In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tqdm import tqdm

# 1. Carga los datos preprocesados
train = pd.read_csv('Data/train_preprocessed.csv')
test = pd.read_csv('Data/test_preprocessed.csv')

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tqdm import tqdm

# 1. Carga los datos preprocesados
train = pd.read_csv('Data/train_preprocessed.csv')
test = pd.read_csv('Data/test_preprocessed.csv')

# 2. Separa variables predictoras y objetivo
y = train['RENDIMIENTO_GLOBAL']
X = train.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
X_test = test.drop(['ID'], axis=1)

# 3. Cross-validation con barra de progreso
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
for train_idx, val_idx in tqdm(cv.split(X, y), total=cv.get_n_splits(), desc="Validación cruzada"):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_val, y_val))
print("CV Accuracy:", sum(scores)/len(scores))

# 4. Entrena el modelo final y predice sobre el test
clf = RandomForestClassifier(n_estimators=100, random_state=42)
with tqdm(total=1, desc="Entrenando modelo final") as pbar:
    clf.fit(X, y)
    pbar.update(1)

y_pred = clf.predict(X_test)

# 5. Guarda el resultado para submission
submission = pd.DataFrame({'ID': test['ID'], 'RENDIMIENTO_GLOBAL': y_pred})
submission.to_csv('submission.csv', index=False)
print("Archivo 'submission.csv' generado.")

NameError: name 'train_test_split' is not defined

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

# 1. Carga los datos preprocesados
train = pd.read_csv('Data/train_preprocessed.csv')
test = pd.read_csv('Data/test_preprocessed.csv')

# 2. Separa variables predictoras y objetivo
y = train['RENDIMIENTO_GLOBAL']
X = train.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
X_test = test.drop(['ID'], axis=1)

# 3. Cross-validation con barra de progreso y medición de tiempo
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
y_true = []
y_pred_cv = []

print("Iniciando validación cruzada...")
start_time = time.time()
for train_idx, val_idx in tqdm(cv.split(X, y), total=cv.get_n_splits(), desc="Validación cruzada"):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    score = clf.score(X_val, y_val)
    scores.append(score)
    # Para métricas detalladas:
    y_pred_fold = clf.predict(X_val)
    y_true.extend(y_val)
    y_pred_cv.extend(y_pred_fold)
elapsed = time.time() - start_time
print(f"Validación cruzada completada en {elapsed/60:.2f} minutos.")
print(f"CV Accuracy promedio: {np.mean(scores):.4f}")

# 4. Entrena el modelo final y predice sobre el test con barra de progreso
print("Entrenando modelo final con todos los datos...")
start_time = time.time()
with tqdm(total=1, desc="Entrenando modelo final") as pbar:
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X, y)
    pbar.update(1)
elapsed_final = time.time() - start_time
print(f"Entrenamiento final completado en {elapsed_final:.2f} segundos.")

y_pred = clf.predict(X_test)

# 5. Guarda el resultado para submission
submission = pd.DataFrame({'ID': test['ID'], 'RENDIMIENTO_GLOBAL': y_pred})
submission.to_csv('submission.csv', index=False)
print("Archivo 'submission.csv' generado.")

# 6. Resultados finales y explicación

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred_cv)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.title('Matriz de confusión (validación cruzada)')
plt.show()

# Reporte de clasificación
print("Reporte de clasificación (validación cruzada):")
print(classification_report(y_true, y_pred_cv))

# Importancia de variables
importances = clf.feature_importances_
features = X.columns
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10,6))
plt.title("Importancia de variables (Random Forest)")
plt.bar(range(len(features)), importances[indices])
plt.xticks(range(len(features)), features[indices], rotation=90)
plt.tight_layout()
plt.show()

# Explicación de los resultados:
print("""
Explicación de los resultados:
- La barra de progreso muestra el avance de la validación cruzada y el entrenamiento final.
- 'CV Accuracy promedio' es la precisión media obtenida en los 5 folds de validación cruzada.
- La matriz de confusión muestra cómo se distribuyen los aciertos y errores por clase.
- El reporte de clasificación incluye precisión, recall y F1-score para cada clase.
- El gráfico de importancia de variables te ayuda a identificar qué variables son más relevantes para el modelo.
- El archivo 'submission.csv' contiene las predicciones finales para el conjunto de test.
""")

Iniciando validación cruzada...


Validación cruzada:   0%|          | 0/5 [01:05<?, ?it/s]


KeyboardInterrupt: 